In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/DAR_Deep Learning Model/MobileVGG.py"

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class MobileNetVGG(nn.Module):
    def __init__(self, alpha=1, num_classes=10):
        super(MobileNetVGG, self).__init__()
        """ Input and 3x3 conv 64 filters"""
        self.front = self.ConvBlock(3, int(64*alpha), 3, 1, dropout=False)

        self.model = nn.Sequential(
            #""" 3x3 conv 64 filters and maxpooling by 2"""
            self.ResidualBlock(int(64*alpha), int(64*alpha), kernel_size=3, stride=2),
            #""" 3x3 conv 128 filters"""
            self.ResidualBlock(int(64*alpha), int(128*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 128 filters and maxpooling by 2"""
            self.ResidualBlock(int(128*alpha), int(128*alpha), kernel_size=3, stride=2),
            #""" 3x3 conv 256 filters"""
            self.ResidualBlock(int(128*alpha), int(256*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 256 filters"""
            self.ResidualBlock(int(256*alpha), int(256*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 256 filters and maxpooling by 2"""
            self.ResidualBlock(int(256*alpha), int(256*alpha), kernel_size=3, stride=2, dropout=True, drop_value=0.05),
            #""" 3x3 conv 512 filters"""
            self.ResidualBlock(int(256*alpha), int(512*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 512 filters"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 512 filters and maxpooling by 2"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=3, stride=2, dropout=True, drop_value=0.1),
            #""" 3x3 conv 512 filters"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 512 filters"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=3, stride=1),
            #""" 3x3 conv 512 filters and maxpooling by 2"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=3, stride=2, dropout=True, drop_value=0.2),
            #""" 7x7 conv 512 filters"""
            self.ResidualBlock(int(512*alpha), int(512*alpha), kernel_size=7, stride=1, dropout=True, drop_value=0.3),
            #""" 7x7 conv 512 filters"""
            self.ConvBlock(int(512*alpha), int(512*alpha), 1, 1, dropout=True, drop_value=0.4),

        )

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        # Classifier
        self.fc = nn.Linear(int(512 * alpha), num_classes)

    def forward(self, x):
        x = self.front(x)
        x = self.model(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x


    def ResidualBlock(self, in_channels, out_channels, kernel_size, stride, dropout=False, drop_value=0.05):
        """Depthwise separable convolution"""
        layers = [
          ('depthwise', nn.Conv2d(in_channels, in_channels, kernel_size, stride,  padding=kernel_size//2, groups=in_channels, bias=False)),
          ('batchnorm1', nn.BatchNorm2d(in_channels)),
          ('leakyrelu1', nn.LeakyReLU(negative_slope=0.01)),

          ('pointwise', nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)),
          ('batchnorm2', nn.BatchNorm2d(out_channels)),
          ('leakyrelu2', nn.LeakyReLU(negative_slope=0.01)),
        ]

        if dropout:
          layers.append(('dropout', nn.Dropout(drop_value)))

        return nn.Sequential(OrderedDict(layers))

    def ConvBlock(self, in_channels, out_channels, kernel_size, stride, dropout=False, drop_value=0.4):
        """Standard convolution block"""
        layers = [
            ('conv', nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=kernel_size//2, bias=False)),
            ('batchnorm', nn.BatchNorm2d(out_channels)),
            ('leakyrelu', nn.LeakyReLU(negative_slope=0.01)),
        ]

        if dropout:
            layers.append(('dropout', nn.Dropout(drop_value)))

        return nn.Sequential(OrderedDict(layers))

Overwriting /content/drive/MyDrive/DAR_Deep Learning Model/MobileVGG.py


In [ ]:
model = MobileNetVGG(alpha=1, num_classes=10)

In [ ]:
print(model)

MobileNetVGG(
  (front): Sequential(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (model): Sequential(
    (0): Sequential(
      (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), groups=64, bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.01)
      (pointwise): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.01